
<p align="center"><img src="https://github.com/Reece323/air-scrape/blob/main/assets/airbnb_logo.png?raw=true" width="300"/></p>
<center> <h1> Analyzing AirBnB Listings </h1> </center>



### This project focuses on data gathered from AirBnB listings
#### Goals:


*   *Manipulating and Exploring the data*
*   *Creating Visualizations*
*   *Use Machine Learning*
*   *Determine which features attracts consumers*
*   *See how to increase revenue as `host`*

In [1]:
# Imports
import pandas as pd
import numpy as np
import json
import urllib.request
from operator import itemgetter
from pprint import pprint
from itertools import chain
import re
import html
# !pip install rich
from rich.console import Console

console = Console()
print = console.print

pd.options.display.max_columns = 4000

import folium
import contextily as cx
import geopandas
import matplotlib.pyplot as plt
from shapely.geometry import Point
from geopandas import GeoDataFrame
import matplotlib as mpl
from matplotlib import cm
from colorspacious import cspace_converter

/Users/codyreece/Desktop/Repos/airbnb/air-scrape/venv/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
url = 'https://raw.githubusercontent.com/Reece323/air-scrape/main/AirBnB/Data/Bentonville_all.json'
response = urllib.request.urlopen(url)
Bentonville = json.loads(response.read())
Bville = pd.json_normalize(Bentonville)

In [3]:
cols_to_drop_temp = [
    # 'url',
    "calendar",
    'rootAmenitySections',
    'p3SummaryAddress',
    'primaryHost.about',
    'primaryHost.badges',
    'primaryHost.firstName',
    'primaryHost.id',
    'primaryHost.isSuperhost',
    'primaryHost.memberSinceFullStr',
    'primaryHost.name',
    'primaryHost.languages',
    'primaryHost.pictureUrl',
    'primaryHost.responseRateWithoutNa',
    'primaryHost.responseTimeWithoutNa',
    'primaryHost.hasInclusionBadge',
    'primaryHost.pictureLargeUrl',
    'primaryHost.hostIntroTags',
    'primaryHost.hostUrl',
    'primaryHost.listingsCount',
    'primaryHost.totalListingsCount',
    'tierId',
    'hasSpecialOffer',
    'hasWeWorkLocation',
    'hasWeWorkLocation',
    'additionalHosts',
    'hometourRooms',
    'hometourSections',
    'descriptionLocale',
    'initialDescriptionAuthorType',
    'localizedCheckInTimeWindow',
    'localizedCheckOutTime',
    'hometourRooms',
    'countryCode',
    'hasHostGuidebook',
    'hasLocalAttractions',
    'neighborhoodCommunityTags',
    'state',
    'paidGrowthRemarketingListingIds',
    'hasCommercialHostInfo',
    'hostSignatureFont.url',
    'nearbyAirportDistanceDescriptions',
    'propertyTypeInCity',
    'renderTierId',
    'isHotel',
    'isNewListing',
    'showReviewTag',
    'isRepresentativeInventory',
    'localizedCity',
    'highlights',
    'highlightsImpressionId',
    'pointOfInterests',
    'hostGuidebook.guidebookUrl',
    'hostGuidebook.localizedNameForHomesPdp',
    'hostGuidebook.title',
    'hostGuidebook.id',
    'pageViewType',
    'previewTags',
    'seeAllHometourSections',
    'enableHighlightsVoting',
    'heroModule.categorizedPhotos',
    'sortedReviews',
    'documentDisplayPictures',
    'sections',
    'p3ImpressionId',
    'paidGrowthRemarketingListingIdsStr',
    'isBusinessTravelReady',
    'address',
    'hasHouseRules',
    'reviewsOrder',
    'hostQuote',
    'localizedListingExpectations',
    'pricing.rate.amount_formatted',
    'pricing.rate.currency',
    'pricing.rate.is_micros_accuracy',
    'pricing.rate_with_service_fee.amount_formatted',
    'pricing.rate_with_service_fee.currency',
    'pricing.rate_with_service_fee.is_micros_accuracy',
    'reviewDetailsInterface.reviewSummary',
    'reviewsModule.appreciationTags',
    'pricing.rate_with_service_fee.amount',
    'pricing.rate_type',
    'sectionedDescription.authorType',
    'sectionedDescription.locale',
    'sectionedDescription.localizedLanguageName',
    'sectionedDescription.notes',
    'sectionedDescription.space',
    'guestControls.allowsNonChinaUsers',
    'guestControls.structuredHouseRulesWithTips',
    'sectionedDescription.neighborhoodOverview',
    'sectionedDescription.access',
    'sectionedDescription.description',
    'sectionedDescription.interaction',
    'sectionedDescription.name',
    'sectionedDescription.summary',
    'sectionedDescription.transit',
    # 'name',
    'listingExpectations',
    'sectionedDescription.houseRules',
    'additionalHouseRules',
    'idStr'
]

Bville = Bville.drop(cols_to_drop_temp, axis=1)

In [4]:
df = Bville.reindex(columns=[
    'url',
    'name',
    'city',
    'country',
    'id',
    'location.lat',
    'location.lng',
    'numberOfGuests',
    'listingRooms',
    'roomType',
    'roomTypeCategory',
    'bedroomLabel',
    'bedLabel',
    'bathroomLabel',
    'listingAmenities',
    'seeAllAmenitySections',
    'categorizedPreviewAmenities',
    'stars',
    'reviewsModule.localizedOverallRating',
    'reviewDetailsInterface.reviewCount',
    'reviews',
    'isHostedBySuperhost',
    'maxNights',
    'minNights',
    'priceDetails',
    'pricing.rate.amount',
    'guestControls.allowsChildren',
    'guestControls.allowsEvents',
    'guestControls.allowsInfants',
    'guestControls.allowsPets',
    'guestControls.allowsSmoking',
    'guestControls.personCapacity',
    'photos',
    'guestControls.structuredHouseRules',
]
)


In [5]:
#function to check dtypes and % of unique values
def checking(df_to_check):
    print(f"Cleaned data type: {df_to_check.dtype}")
    print(f'% of listings with # unique: \n{df_to_check.value_counts(normalize=True)}\n')

In [6]:
# 'rooms'
df['bedroomLabel'] = df['bedroomLabel'].replace('Studio', '0 bedrooms')
new = df["bathroomLabel"].str.split(" ", n = 1, expand = True)
# print(new)
# print(new[0])
# making separate column from new data frame
df["rooms"] = new[0]
# checking 'rooms'
checking(df.rooms)


new1 = df["bedLabel"].str.split(" ", n = 1, expand = True)
# making separate column from new data frame
df["beds"] = new1[0]
# checking 'rooms'
checking(df.beds)


new = df["bathroomLabel"].str.split(" ", n = 1, expand = True)
df['baths'] = new[0].astype(float)
checking(df.baths)

Cleaned data type: object

% of listings with # unique: 
1      0.536765
2      0.308824
1.5    0.080882
2.5    0.036765
3      0.033088
3.5    0.003676
Name: rooms, dtype: float64

Cleaned data type: object

% of listings with # unique: 
2    0.301471
3    0.272059
1    0.191176
4    0.176471
5    0.025735
6    0.014706
7    0.007353
9    0.007353
0    0.003676
Name: beds, dtype: float64

Cleaned data type: float64

% of listings with # unique: 
1.0    0.536765
2.0    0.308824
1.5    0.080882
2.5    0.036765
3.0    0.033088
3.5    0.003676
Name: baths, dtype: float64

In [7]:
df.columns = [re.sub("[ ,\$]", "_", re.sub("[.]", "", c)) for c in df.columns]

df['geometry'] = [(xy) for xy in zip(df.locationlat, df.locationlng)]

In [8]:
print(df.columns)
df

Index(['url', 'name', 'city', 'country', 'id', 'locationlat', 'locationlng',
       'numberOfGuests', 'listingRooms', 'roomType', 'roomTypeCategory',
       'bedroomLabel', 'bedLabel', 'bathroomLabel', 'listingAmenities',
       'seeAllAmenitySections', 'categorizedPreviewAmenities', 'stars',
       'reviewsModulelocalizedOverallRating',
       'reviewDetailsInterfacereviewCount', 'reviews', 'isHostedBySuperhost',
       'maxNights', 'minNights', 'priceDetails', 'pricingrateamount',
       'guestControlsallowsChildren', 'guestControlsallowsEvents',
       'guestControlsallowsInfants', 'guestControlsallowsPets',
       'guestControlsallowsSmoking', 'guestControlspersonCapacity', 'photos',
       'guestControlsstructuredHouseRules', 'rooms', 'beds', 'baths',
       'geometry'],
      dtype='object')

,url,name,city,country,id,locationlat,locationlng,numberOfGuests,listingRooms,roomType,roomTypeCategory,bedroomLabel,bedLabel,bathroomLabel,listingAmenities,seeAllAmenitySections,categorizedPreviewAmenities,stars,reviewsModulelocalizedOverallRating,reviewDetailsInterfacereviewCount,reviews,isHostedBySuperhost,maxNights,minNights,priceDetails,pricingrateamount,guestControlsallowsChildren,guestControlsallowsEvents,guestControlsallowsInfants,guestControlsallowsPets,guestControlsallowsSmoking,guestControlspersonCapacity,photos,guestControlsstructuredHouseRules,rooms,beds,baths,geometry
0,https://www.airbnb.com/rooms/32721035,Cozy & Relaxing Suite: Pool/Movie Screen/Trails,Bentonville,United States,32721035,36.40187,-94.22518,4,"[{'beds': [{'id': '37190702/queen_bed', 'quant...",Entire guest suite,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.96,366,"[{'author': {'firstName': 'Zach', 'hasProfileP...",True,29,2,"[{'label': 'Cleaning Fee', 'value': '$20', 'at...",49,True,False,True,False,False,4,[{'caption': 'Relax poolside after a day of ex...,"[No pets, No smoking, parties, or events, Chec...",1,2,1.0,"(36.40187, -94.22518)"
1,https://www.airbnb.com/rooms/14023239,"Cozy Garden Retreat in Downtown - Patio, Bike ...",Bentonville,United States,14023239,36.37226,-94.21269,4,"[{'beds': [{'id': '8053552/sofa_bed', 'quantit...",Entire guesthouse,entire_home,1 bedroom,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.97,329,"[{'author': {'firstName': 'Savannah', 'hasProf...",True,14,1,"[{'label': 'Cleaning Fee', 'value': '$50', 'at...",92,True,False,True,False,False,4,"[{'caption': 'A cozy, comfortable, light-fille...","[No pets, No smoking, parties, or events, Chec...",1,2,1.0,"(36.37226, -94.21269)"
2,https://www.airbnb.com/rooms/38783656,Memorial Park Hideaway - near Crystal Bridges,Bentonville,United States,38783656,36.37763,-94.18872,2,"[{'beds': [{'id': '65625913/king_bed', 'quanti...",Private room in residential home,private_room,1 bedroom,1 bed,1 private bath,"[{'description': '', 'id': 8, 'isPresent': Fal...","[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Wifi, ...",5.0,4.92,239,"[{'author': {'firstName': 'Rachel', 'hasProfil...",True,28,2,"[{'label': 'Cleaning Fee', 'value': '$10', 'at...",56,True,False,False,False,False,2,"[{'caption': '', 'id': 1106743335, 'large': 'h...",[Not safe or suitable for infants (under 2 yea...,1,1,1.0,"(36.37763, -94.18872)"
3,https://www.airbnb.com/rooms/45565472,Studio w/Deck (Private),Bentonville,United States,45565472,36.37142,-94.22504,4,"[{'beds': [{'id': '107302929/king_bed', 'quant...",Entire residential home,entire_home,0 bedrooms,2 beds,1 bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",5.0,4.97,67,"[{'author': {'firstName': 'Jessica', 'hasProfi...",True,1125,2,"[{'label': 'Cleaning Fee', 'value': '$25', 'at...",89,True,False,True,False,False,4,"[{'caption': 'Sofa bed, king bed', 'id': 10926...","[No pets, No smoking, parties, or events, Chec...",1,2,1.0,"(36.37142, -94.22504)"
4,https://www.airbnb.com/rooms/45831904,Spacious private bedroom located close to down...,Bentonville,United States,45831904,36.36268,-94.18553,2,"[{'beds': [{'id': '110242780/couch', 'quantity...",Private room in residential home,private_room,1 bedroom,1 bed,1 shared bath,[{'description': 'Space where guests can cook ...,"[{'id': 'pdp_m2_5_bathroom', 'title': 'Bathroo...","[{'category': 'Standard', 'amenities': 'Kitche...",4.5,4.69,16,"[{'author': {'firstName': 'Jiaheng', 'hasProfi...",True,365,2,"[{'label': 'Extra people', 'value': 'No Charge...",30,True,True,True,True,False,2,"[{'caption': ''

In [9]:
photoss = df['photos'][0]
pics = [d['large'] for d in photoss if 'large' in d]

In [10]:
coords = [36.3729, -94.2086]
lat = list(df['locationlat'])
lon = list(df['locationlng'])
name = list(df['name'])
url = list(df['url'])
rate = list(df['reviewsModulelocalizedOverallRating'])
price = list(df['pricingrateamount'])
guests = list(df.numberOfGuests)
rooms = list(df.rooms)
beds = list(df.beds)
baths = list(df.baths)

In [11]:
map=folium.Map(location=coords,zoom_start=11)
fg=folium.FeatureGroup(name="my map")

def map_listings():
  for lt, ln, nm, site, rater, prc, gsts, bds, bths, in zip(lat, lon, name, url, rate, price, guests, beds, baths):
    fg.add_child(
            folium.Marker(location=[lt, ln],
                          popup='<h5> <center> <b>' + nm + "</b> </center> </h5>\
                            <hr> <div> <b> <center> <h5> $" + str(prc) + "/night </h5> </center> </b> </div>\
                            <div> <center> <h5> <p>&#11088; "+str(rater)+" </h5> </center> </div>\
                            <center> <img width='150'src='https://github.com/Reece323/air-scrape/blob/main/Dash/assets/airbnb_logo.png?raw=true'/></center>\
                            <a href=" + site + ">\
                            <center> <b> <h5> Go to Listing </h5> </b> </center> </a>",
                          icon=folium.Icon(color='darkblue')
                          )
        )

  return map.add_child(fg), 

In [ ]:
map_view = html.Iframe(
                id='map',
                srcDoc=open('Bentonville.html', 'r').read(),
                width='650',
                height='650',
                className='iframo',
                )